In [1]:
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
%run ../prepare_data.py -N_users 0

In [3]:
# Expand cycles so that there is a line per date (active or not) with a boolean indicator of period
try:
    cycles_processed = joblib.load("../data/cycles_processed.pkl.gz")
except:
    cycles_processed = pd.concat([expand_cycle(cycle) for _, cycle in cycles.iterrows()])
    joblib.dump(cycles_processed, "../data/cycles_processed.pkl.gz")

In [4]:
# One hot encode the symptoms
tracking_processed = pd.get_dummies(
    tracking[["user_id", "date", "symptom"]],
    columns=['symptom'], prefix='', prefix_sep=''
)

In [5]:
# Aggregate symptoms per day
tracking_processed = tracking_processed.groupby(['user_id', 'date']).sum()[list_of_symptoms]
training = pd.merge(cycles_processed, tracking_processed, left_index=True, right_index=True, how='outer').fillna(0)

In [6]:
# Find the first day the user started using the app
training = pd.merge(
    training,
    cycles.groupby('user_id').agg({'cycle_start': {'first_use': 'min'}}).reset_index().set_index('user_id')['cycle_start'],
    left_index=True, right_index=True
)

In [7]:
# Find the absolute day for each row from the day the user started using the app
absolute_day = (training.reset_index().date.dt.date - training.reset_index().first_use.dt.date).dt.days + 1
absolute_day.index = training.index
training['absolute_day'] = absolute_day
# Won't need this column anymore
training = training.drop('first_use', axis=1)

In [8]:
# Order the columns
training_columns = list_of_symptoms + ['cycle_id', 'day_in_cycle', 'absolute_day', 'period']
training = training[training_columns]